In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer 

In [4]:
training = pd.read_csv('training.csv')
testing = pd.read_csv('test.csv')

# input -1 for missing values
imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=-1)

# Convert back to dataframe and replace the missing values
training = pd.DataFrame(imputer.fit_transform(training))

# Get the last feature column
lastFeature = training.iloc[:, -1]

# Split the last feature by spaces
splitFeatures = lastFeature.str.split(' ', expand=True)

# Drop the original last feature and concatenate the new split features
training = pd.concat([training.drop(training.columns[-1], axis=1), splitFeatures], axis=1)

# Rename the columns to pixel_i
training.columns = [f'pixel_{i}' for i in range(training.shape[1])]


# Same for testing
testImg = testing.iloc[:, -1]
splitTest = testImg.str.split(' ', expand=True)
testing = pd.concat([testing.drop(testing.columns[-1], axis=1), splitTest], axis=1)

# Rename the columns to pixel_i
testing.columns = ['ImageId'] + [f'pixel_{i}' for i in range(1, testing.shape[1])]


# print(training.head)